[View in Colaboratory](https://colab.research.google.com/github/shalini0289/Colab/blob/master/pipeline_final_twitter_analytics_vidhya.ipynb)

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 473590152623840415]

In [0]:
from google.colab import auth
from googleapiclient.discovery import build
import io , requests
import sys

import pandas as pd
import os
auth.authenticate_user()

In [0]:
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')
SOURCE_FOLDER=''

In [0]:
def get_parent_folder(folder_name):
  page_token = None
  folder_array = []
  query = "name='%s' and mimeType='application/vnd.google-apps.folder'" % folder_name
  while True:
      response = drive_service.files().list(q=query,
                                          spaces='drive',
                                          fields='nextPageToken, files(id, name)',
                                          pageToken=page_token).execute()
      for file in response.get('files', []):
          # Process change
          #print (file.get('name'), file.get('id'))
          folder_array.append({"name" : file.get('name'), "id" : file.get('id')})
      page_token = response.get('nextPageToken', None)
      if page_token is None:
          break
  return folder_array

In [0]:
def get_files_from_parent(parent_id):
  page_token = None
  folder_array = dict()
  query = "'%s' in parents" % parent_id
  while True:
      response = drive_service.files().list(q=query,
                                          spaces='drive',
                                          fields='nextPageToken, files(id, name)',
                                          pageToken=page_token).execute()
      for file in response.get('files', []):
          # Process change
          #print (file.get('name'), file.get('id'))
          folder_array.update({file.get('name'):file.get('id')})
      page_token = response.get('nextPageToken', None)
      if page_token is None:
          break
  return folder_array

In [0]:
def get_file_buffer(file_id, verbose=0):
  from googleapiclient.http import MediaIoBaseDownload
  request = drive_service.files().get_media(fileId=file_id)
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  while done is False:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    progress, done = downloader.next_chunk()
    if verbose:
      sys.stdout.flush()
      sys.stdout.write('\r')
      percentage_done = progress.resumable_progress * 100/progress.total_size
      sys.stdout.write("[%-100s] %d%%" % ('='*int(percentage_done), int(percentage_done)))
  downloaded.seek(0)
  return downloaded

In [8]:
parent_folder = get_parent_folder('av-sa')
print(parent_folder)
parent_folder[0]["id"]

[{'name': 'av-sa', 'id': '1uOG8csXYChpdzul_0v5lPgY-N5ssebG5'}]


'1uOG8csXYChpdzul_0v5lPgY-N5ssebG5'

In [9]:
input_file_meta = get_files_from_parent(parent_folder[0]["id"])
print(input_file_meta)

{'test_tweets_anuFYb8.csv': '1DXtnnPd4THLQj6kUAU2JODxaQd3r1EXh', 'train_E6oV3lV.csv': '1FO1N6elMtXjPAh1Sz6BHojgHS-tqM8U3'}


In [10]:
for file, id in input_file_meta.items():
  downloaded = get_file_buffer(id, verbose=1)
  dest_file = os.path.join(SOURCE_FOLDER, file)
  print("processing %s data" % file)
  with open(dest_file, "wb") as out:
    out.write(downloaded.read())
    print("Done %s" % dest_file)

[====================================================================================================] 100%processing test_tweets_anuFYb8.csv data
Done test_tweets_anuFYb8.csv
[====================================================================================================] 100%processing train_E6oV3lV.csv data
Done train_E6oV3lV.csv


In [0]:
train=pd.read_csv(os.path.join(SOURCE_FOLDER,'train_E6oV3lV.csv'))
test=pd.read_csv(os.path.join(SOURCE_FOLDER,'test_tweets_anuFYb8.csv'))

In [0]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt   

In [13]:
!pip install wordcloud
!pip install gensim


    100% |████████████████████████████████| 368kB 4.8MB/s 
    100% |████████████████████████████████| 23.5MB 989kB/s 
    100% |████████████████████████████████| 1.4MB 9.9MB/s 
    100% |████████████████████████████████| 133kB 28.9MB/s 
    100% |████████████████████████████████| 61kB 23.4MB/s 
    100% |████████████████████████████████| 4.7MB 930kB/s 
    100% |████████████████████████████████| 552kB 24.6MB/s 
  Running setup.py bdist_wheel for smart-open ... - done
  Stored in directory: /root/.cache/pip/wheels/73/f1/9b/ccf93d4ba073b6f79b1ed9df68ab5ce048d8136d0efcf90b30
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [0]:
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
import gensim
from sklearn.linear_model import LogisticRegression
from sklearn import utils
from sklearn.model_selection import train_test_split
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument, LabeledSentence

warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

In [0]:
# remove twitter handles (@user)
# @[\w]* -  a regular expression which will pick any word starting with ‘@

train['tidy_tweet'] = np.vectorize(remove_pattern)(train['tweet'], "@[\w]*")
test['tidy_tweet'] = np.vectorize(remove_pattern)(test['tweet'], "@[\w]*")

In [0]:
# remove special characters, numbers, punctuations
# replace everything except characters and hashtags with spaces

#train['tidy_tweet'] = train['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")
#test['tidy_tweet'] = test['tidy_tweet'].str.replace("[^a-zA-Z#]", " ")

In [0]:
import re
## example ## 
re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", " ","ouch...junior is angryð#got7 #junior #yugyo..., @user")
def process_tweet(tweet):
    return " ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", " ",tweet.lower()).split())

In [0]:
train['tidy_tweet'] = train['tidy_tweet'].apply(process_tweet)
test['tidy_tweet'] = test['tidy_tweet'].apply(process_tweet)

In [0]:
#remove short words like : oh, hm etc

train['tidy_tweet'] = train['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
test['tidy_tweet'] = test['tidy_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [20]:
# splitting a string of text into tokens(individual terms or words)

train_tokenized_tweet = train['tidy_tweet'].apply(lambda x: x.split())
train_tokenized_tweet.head()

0    [when, father, dysfunctional, selfish, drags, ...
1    [thanks, lyft, credit, cause, they, offer, whe...
2                              [bihday, your, majesty]
3                      [model, love, take, with, time]
4                    [factsguide, society, motivation]
Name: tidy_tweet, dtype: object

In [21]:
# splitting a string of text into tokens(individual terms or words)

test_tokenized_tweet = test['tidy_tweet'].apply(lambda x: x.split())
test_tokenized_tweet.head()

0    [studiolife, aislife, requires, passion, dedic...
1    [white, supremacists, want, everyone, birds, m...
2    [safe, ways, heal, your, acne, altwaystoheal, ...
3    [cursed, child, book, reservations, already, w...
4    [bihday, amazing, hilarious, nephew, ahmir, un...
Name: tidy_tweet, dtype: object

In [0]:
# Stemming is a rule-based process of stripping the suffixes (“ing”, “ly”, “es”, “s” etc)

from nltk.stem.porter import *
stemmer = PorterStemmer()

train_tokenized_tweet = train_tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
test_tokenized_tweet = test_tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming

In [0]:
#combine tokens
for i in range(len(train_tokenized_tweet)):
 train_tokenized_tweet[i] = ",".join(train_tokenized_tweet[i])
train['tidy_tweet'] = train_tokenized_tweet

for i in range(len(test_tokenized_tweet)):
 test_tokenized_tweet[i] = ",".join(test_tokenized_tweet[i])
test['tidy_tweet'] = test_tokenized_tweet

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train["tidy_tweet"],train["label"], test_size = 0.2, random_state = 3)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score


In [0]:
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
                     

In [0]:
text_clf = text_clf.fit(train["tidy_tweet"], train["label"])

In [0]:
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],'tfidf__use_idf': (True, False),'clf__alpha': (1e-2, 1e-3)}
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(train["tidy_tweet"], train["label"])


In [0]:
from sklearn.metrics import confusion_matrix,f1_score
prediction = gs_clf.predict(test["tidy_tweet"])

In [36]:
prediction

array([0, 0, 0, ..., 0, 0, 0])

In [0]:
test['label'] = prediction
submission = test[['id','label']]
my_sub = os.path.join(SOURCE_FOLDER, "sub_tfdif.csv")
submission.to_csv(my_sub, index=False) # writing data to a CSV file

In [0]:
from google.colab import files
files.download( "sub_tfdif.csv" )